<a href="https://colab.research.google.com/github/RiverGumSecurity/AILabs/blob/main/022_LargeLanguageModels/inference.ipynb" target="_new"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This code fixes a word wrapping issue
# in Google Co-Lab on Text Streamer output.
from warnings import warn
from IPython import get_ipython
from IPython.display import HTML, display
from transformers import TextStreamer

def enable_word_wrap(*args, **kwargs):
    display(HTML('''
    <style>
        pre {
            white-space: pre-wrap;
        }
    </style>
    '''))

def register_ipython_callback_once(event_name, cb):
    ev = get_ipython().events
    cb_unregs = [cb_old for cb_old in ev.callbacks[event_name] if cb_old.__name__ == cb.__name__]
    if len(cb_unregs) == 1 and cb.__code__ == cb_unregs[0].__code__:
        return
    for cb_old in cb_unregs:
        warn(f'Removing unexpected callback {cb_old}.')
        ev.unregister(event_name, cb_old)
    ev.register(event_name, cb)

register_ipython_callback_once('pre_run_cell', enable_word_wrap)

In [ ]:
import torch
import warnings
import os
import sys
import pathlib
try:
    import unsloth
except:
    if 'google.colab' in sys.modules:
        !pip install unsloth

# suppresses some noisey warnings which are just annoying
warnings.filterwarnings('ignore')
max_seq_length = 4096

# Setup Hugging Face Credentials.
HF_APIKEY = ''
if 'google.colab' in sys.modules:
    from google.colab import userdata
    HF_APIKEY = userdata.get('HF_APIKEY')
else:
    with open(pathlib.Path.home() / '.hfkey') as hf:
        HF_APIKEY = hf.read().strip()
if not HF_APIKEY:
    print('[-] ERROR: Cannot continue without Hugging Face API Key')
    sys.exit(0)
os.environ['HF_TOKEN'] = HF_APIKEY

model, tokenizer = unsloth.FastLanguageModel.from_pretrained(
    model_name = "joffthyer/l3-bhisblogs-infotech-improved",
    max_seq_length = max_seq_length,
    dtype = None, load_in_4bit = True
)

In [ ]:
from transformers import TextStreamer

prompt = """
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [ ]:
_ = unsloth.FastLanguageModel.for_inference(model)

def query_model(text):
    inputs = tokenizer(
        [prompt.format(text, '', '')], return_tensors = "pt").to("cuda")
    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

query_model('Write a blog in the style of Joff Thyer about password policies and encryption.')

